BTM

# <center> Capstone Project - Neighborhoods businesses Comparison Using K-Means Clustering, Case of New York vs Toronto (Week 2)</center>
### Applied Data Science Capstone by IBM 

##  Table of Content
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)
* [References](#References)



## Introduction: Business Problem <a name="introduction"></a>

### Background
Cities around the World contain certain number of neighborhoods formed of venues naturally distributed in many different way.  The historic need to identify city’s driver factors such as financial incentives, quality of urbanisation, social and food diversities, crime level, housing, education, health, etc, often lead to the exploration of similarity or dissimilarity between cities. 
Comparing venues within neighborhoods in cities helps the orientation of decision making in terms of business investment, immigration or relocation, job hunting and much more.  The analysis in this work is applied to the two most populous and diverse cities of the United State and Canada: New York vs Toronto.

### Problem Definition 
The Neighborhoods businesses comparison of two or more different cities is a type of unsupervised classification where venues have to be segmented and grouped into a certain number of dissimilar and non-overlapping clusters.  These clusters should contain similar venues of common characteristics, without any internal structure or label.  K-Means algorithms is one of the most popular tools of segmentation of unsupervised data that will be used here coupled with foursquare API to make essentially venues calls to retrieve needed information. 

### Stakeholders
This work is particularly targeting business people who need to invest by giving them insight views of market in neighborhoods.  This work concern also at large job seekers to explore possibilities that offer one city over the other.  The results of this work should also advice on people movement, immigration and relocation, because positive numbers on neighborhoods businesses in a city, will most probably influence the influx of people.



## Data <a name="data"></a>

### Data Source

Data used in this work were collected from open sources on the internet.  
For New York, a total of 306 neighborhoods with a total of 5 boroughs in neighborhood as well as the latitude and the longitude of each neighborhood fount on the web under the link: https://geo.nyu.edu/catalog/nyu_2451_34572

For Toronto, City neighborhoods were found on Wikipedia website under the following link:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

This link provided postal codes of venues that could be transform into needed data by using pandas library function.  In case the geocoder package couldn't work, the geographical coordinates of postal code can be found under the link:
http://cocl.us/Geospatial_data
These data will be cleaned and formatted accordingly to adequately feed the k-Means algorithm for convergent clustering.


The necessary lxml data of Toronto City neighborhoods were found on Wikipedia website under the following link: 

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

This link provided postal codes of venues that could be transform into needed data by using pandas library function.

In case the geocoder package couldn't work, the geographical coordinates of postal code can be found under the link:

http://cocl.us/Geospatial_data

## 1.3 Downoad of Raw Data 

#### Importing Useful Libraries

In [1]:
# For Python location of important packages
from IPython.core.magic import register_line_magic
import sys
sys.path
import os

# !pip install --upgrade distribute
# !pip install --upgrade setuptools

# !python -m pip install -U pip

# library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup

# import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip3 install wget  # Installation of wget for reading webs URL
import wget

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

ERROR: Invalid requirement: '#'


Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Scraping of Raw Data on Toronto

In [2]:
# Downloading of Data
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])

# Show the top 
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Removing "\n" in all columns and display the 12 rows of the dataset

In [3]:
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")

df = df[df.Neighborhood !='Not assigned'].reset_index() # Cleaning all rows with "Not assigned" in Neighborhood column

df.head(12)

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


Grouping Neighborhoods having the same Postal Code

In [4]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()

df.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
# Checking the size of the obtained dataset
print("Shape:", df.shape)

Shape: (103, 3)


### Toronto Geographical Coordinates for each Neighborhood

Because the Geocoder couldn't generate geo-coordinates, we used the csv file

In [6]:
df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.head(12)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [7]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Exploration and Clustering of Neighborhoods in Toronto

#####  Generation of Toronto Latitude and Longitude

In [8]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


#### Visualization of Toronto Map and Neighborhood Markers

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

# Map with Neighborhoods marked

for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### New York Download and Explore Dataset

#### Loading and Exploring of New York Dataset

In [41]:
import wget
url = 'https://cocl.us/new_york_dataset' 
new_york_dataset = wget.download(url)

print('Data downloaded!')

100% [............................................................................] 115774 / 115774Data downloaded!


In [47]:
import json
with open('new_york_dataset') as json_data:
    newyork_data = json.load(json_data)

newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [48]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [49]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [50]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [51]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [52]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [53]:
neighborhoods.head(12)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


It is found that the dataset has all 5 boroughs and 306 neighborhoods.

In [54]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
      )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [55]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [58]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.


## Methodology <a name="methodology"></a>

Having neighborhoods of Toronto and New York, to find venues in these neighborhoods, Foursquare API is used. With explored, k-Means is used to segment venues into clusters. By comparing similar clusters of the two city, we will be able to conclude on the level of business activities into the city; this is not to confuse with the number of enterprises in each city.

###  Using Foursquare API

My Foursquare Credentials, hiden to public on GitHub repository

In [59]:
# @hidden_cell
CLIENT_ID = 'KNWPZDKIHCJVP3WR4EXNDSFHF1CIOEOFXAZJP5LEBPA4YZMR' # your Foursquare ID
CLIENT_SECRET = 'JVBQSID1P1YJ5FF1EH2VEDXVKX0M5DMWNESJLL41FHAL5ZP4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### On Toronto
Let explore the first neighborhood in the dataframe 'df_toronto'

In [60]:
neighborhood_name = df_toronto_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


Neighborhood Coordinates

In [62]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 1000 venues that are in The Beaches within a radius of 10000 meters.

In [64]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

#### Extracting venue cstegories

In [65]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Cleaning of Json dataframe

In [66]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Morningside Park,Park,43.786546,-79.205322
5,Gorilla Exhibit,Zoo Exhibit,43.819080,-79.184235
6,"Lamanna's Bakery, Cafe & Fine Foods",Bakery,43.797971,-79.148432
7,Orangutan Exhibit,Zoo Exhibit,43.818413,-79.182548
8,Australasia Pavillion,Zoo Exhibit,43.822563,-79.183286
9,Mona's Roti,Caribbean Restaurant,43.791613,-79.251015


### Exploration of Neighborhood in Toronto City

Considering an zone in a radius of 10 km

In [68]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Looping the function on each venue to generate a new dataframe named toronto_venues

In [69]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,"Malvern, Rouge",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,"Malvern, Rouge",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
3,"Malvern, Rouge",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
4,"Malvern, Rouge",43.806686,-79.194353,Morningside Park,43.786546,-79.205322,Park
5,"Malvern, Rouge",43.806686,-79.194353,Gorilla Exhibit,43.819080,-79.184235,Zoo Exhibit
6,"Malvern, Rouge",43.806686,-79.194353,"Lamanna's Bakery, Cafe & Fine Foods",43.797971,-79.148432,Bakery
7,"Malvern, Rouge",43.806686,-79.194353,Orangutan Exhibit,43.818413,-79.182548,Zoo Exhibit
8,"Malvern, Rouge",43.806686,-79.194353,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit
9,"Malvern, Rouge",43.806686,-79.194353,Mona's Roti,43.791613,-79.251015,Caribbean Restaurant


#### Number of venues returned for each Neighborhood

In [70]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Wilson Heights, Downsview North",100,100,100,100,100,100
Bayview Village,100,100,100,100,100,100
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


#### Number of unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [72]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(12)

,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Botanical Garden,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plaza,Pool Hall,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Road,Rock Climbing Spot,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [73]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(12)

,Neighborhood,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Botanical Garden,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plaza,Pool Hall,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Road,Rock Climbing Spot,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Zoo
0,Agincourt,0.04,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.0,0.0,0.01,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.0,0.01,0.00,0.0,0.02,0.01,0.00,0.00,0.01,0.02,0.00,0.01,0.01,0.00,0.0,0.0,0.05,0.00,0.02,0.0,0.00,0.00,0.01,0.00,0.02,0.00,0.0,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.0,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.01,0.02,0.00,0.01,0.03,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.02,0.0,0.03,0.00,0.00,0.01,0.02,0.0,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.01,0.01,0.06,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.02,0.00,0.00,0.0,0.00,0.0,0.04,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.02,0.01,0.0,0.02,0.00,0.00,0.0,0.0,0.01,0.01,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.01,0.00,0.0,0.00,0.01,0.0,0.00,0.06,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0

#### Checking the 10 most common venues in each neighborhood.

In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Middle Eastern Restaurant,Supermarket,Caribbean Restaurant,Zoo Exhibit,Park,Liquor Store,Grocery Store,Indian Restaurant,Japanese Restaurant,Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Park,Bakery,Burger Joint,Seafood Restaurant,Liquor Store,Grocery Store,Pizza Place,Sporting Goods Shop,Ice Cream Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Park,Italian Restaurant,Middle Eastern Restaurant,Café,Clothing Store,Korean Restaurant,Supermarket,Furniture / Home Store,Grocery Store
3,Bayview Village,Coffee Shop,Middle Eastern Restaurant,Bakery,Japanese Restaurant,Chinese Restaurant,Bagel Shop,Park,Dessert Shop,Supermarket,Korean Restaurant
4,"Bedford Park, Lawrence Manor East",Park,Italian Restaurant,Café,Coffee Shop,Bakery,Liquor Store,Restaurant,Hotel,Dessert Shop,Furniture / Home Store
5,Berczy Park,Coffee Shop,Park,Café,Bakery,Diner,Hotel,Sandwich Place,Vegetarian / Vegan Restaurant,Restaurant,Historic Site
6,"Birch Cliff, Cliffside West",Park,Coffee Shop,Café,Beach,Brewery,Farmers Market,Grocery Store,Bakery,Pizza Place,Caribbean Restaurant
7,"Brockton, Parkdale Village, Exhibition Place",Park,Café,Brewery,Restaurant,Bakery,Pizza Place,Bar,Beer Bar,Italian Restaurant,Sandwich Place
8,"Business reply mail Processing Centre, South C...",Coffee Shop,Park,Café,Beach,Farmers Market,Gastropub,Historic Site,Brewery,Japanese Restaurant,Bakery
9,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Coffee Shop,Café,Bakery,Restaurant,Beer Bar,Sandwich Place,Hotel,Diner,Japanese Restaurant



## Analysis <a name="analysis"></a>

##### Running k-Means algorithm to group neighborhood into clusters. Let take k = 5

In [75]:
# set number of clusters = 5
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 2, 2, 1, 3, 3, 1, 3, 3])

#### New dataframe of top 10 venues for each neighborhood

In [105]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_,'false')

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on ='Neighborhood')

# Drop of duplicate column
toronto_merged = toronto_merged.T.drop_duplicates().T

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944,0,Zoo Exhibit,Park,Supermarket,Ice Cream Shop,Caribbean Restaurant,Pharmacy,Grocery Store,Burger Joint,Breakfast Spot,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605,0,Coffee Shop,Park,Ice Cream Shop,Indian Restaurant,Zoo Exhibit,Supermarket,Beach,Caribbean Restaurant,Gym,Liquor Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887,0,Park,Zoo Exhibit,Caribbean Restaurant,Burger Joint,Grocery Store,Coffee Shop,Breakfast Spot,Beach,Supermarket,Bookstore
3,M1G,Scarborough,Woburn,43.771,-79.2169,0,Park,Caribbean Restaurant,Indian Restaurant,Zoo Exhibit,Burger Joint,Grocery Store,Middle Eastern Restaurant,Supermarket,Coffee Shop,Bakery
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,0,Caribbean Restaurant,Indian Restaurant,Park,Middle Eastern Restaurant,Bakery,Grocery Store,Gym,Supermarket,Coffee Shop,Burger Joint


#### Finally, let's visualize the resulting clusters 

In [138]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(
        toronto_merged['Latitude'], 
        toronto_merged['Longitude'], 
        toronto_merged['Neighborhood'], 
        toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
       fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###  Toronto Neighborhoods Clustering Examination

Exam of each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Ctuster 1

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Zoo Exhibit,Park,Supermarket,Ice Cream Shop,Caribbean Restaurant,Pharmacy,Grocery Store,Burger Joint,Breakfast Spot,Fast Food Restaurant
1,Scarborough,0,Coffee Shop,Park,Ice Cream Shop,Indian Restaurant,Zoo Exhibit,Supermarket,Beach,Caribbean Restaurant,Gym,Liquor Store
2,Scarborough,0,Park,Zoo Exhibit,Caribbean Restaurant,Burger Joint,Grocery Store,Coffee Shop,Breakfast Spot,Beach,Supermarket,Bookstore
3,Scarborough,0,Park,Caribbean Restaurant,Indian Restaurant,Zoo Exhibit,Burger Joint,Grocery Store,Middle Eastern Restaurant,Supermarket,Coffee Shop,Bakery
4,Scarborough,0,Caribbean Restaurant,Indian Restaurant,Park,Middle Eastern Restaurant,Bakery,Grocery Store,Gym,Supermarket,Coffee Shop,Burger Joint
5,Scarborough,0,Park,Caribbean Restaurant,Coffee Shop,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Grocery Store,Ice Cream Shop,Beach,Gym
6,Scarborough,0,Park,Beach,Coffee Shop,Café,Caribbean Restaurant,Burger Joint,Indian Restaurant,Brewery,Middle Eastern Restaurant,Grocery Store
8,Scarborough,0,Park,Beach,Coffee Shop,Indian Restaurant,Brewery,Middle Eastern Restaurant,Burger Joint,Grocery Store,Café,Caribbean Restaurant
10,Scarborough,0,Park,Caribbean Restaurant,Coffee Shop,Supermarket,Burger Joint,Middle Eastern Restaurant,Burrito Place,Indian Restaurant,Grocery Store,Italian Restaurant
11,Scarborough,0,Park,Middle Eastern Restaurant,Caribbean Restaurant,Café,Supermarket,Grocery Store,Indian Restaurant,Breakfast Spot,Liquor Store,Bakery


####  Cluster 2

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,North York,1,Park,Café,Italian Restaurant,Bakery,Coffee Shop,Grocery Store,Brewery,Liquor Store,Middle Eastern Restaurant,Sandwich Place
27,North York,1,Park,Café,Italian Restaurant,Bakery,Coffee Shop,Grocery Store,Brewery,Liquor Store,Middle Eastern Restaurant,Sandwich Place
34,North York,1,Park,Café,Brewery,Beach,Coffee Shop,Grocery Store,Bakery,Middle Eastern Restaurant,Pizza Place,Farmers Market
35,East York,1,Park,Café,Beach,Brewery,Ice Cream Shop,Gastropub,Coffee Shop,Grocery Store,Historic Site,Indian Restaurant
44,Central Toronto,1,Park,Café,Coffee Shop,Hotel,Brewery,Restaurant,Italian Restaurant,Japanese Restaurant,Bakery,Historic Site
45,Central Toronto,1,Park,Café,Coffee Shop,Bakery,Brewery,Restaurant,Italian Restaurant,Japanese Restaurant,Clothing Store,Sandwich Place
46,Central Toronto,1,Park,Café,Coffee Shop,Italian Restaurant,Bakery,Japanese Restaurant,Restaurant,Clothing Store,Historic Site,Hotel
47,Central Toronto,1,Park,Café,Coffee Shop,Brewery,Restaurant,Clothing Store,Bakery,Ice Cream Shop,Sandwich Place,Historic Site
49,Central Toronto,1,Park,Café,Coffee Shop,Restaurant,Gastropub,Brewery,Beer Bar,Sandwich Place,Italian Restaurant,Pizza Place
62,North York,1,Park,Italian Restaurant,Café,Coffee Shop,Bakery,Liquor Store,Restaurant,Hotel,Dessert Shop,Furniture / Home Store


#### Cluster 3

In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,2,Middle Eastern Restaurant,Supermarket,Caribbean Restaurant,Chinese Restaurant,Restaurant,Japanese Restaurant,Liquor Store,Indian Restaurant,Grocery Store,BBQ Joint
14,Scarborough,2,Supermarket,Caribbean Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Park,Noodle House,Bakery,Dessert Shop,Grocery Store,Ice Cream Shop
15,Scarborough,2,Supermarket,Caribbean Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Grocery Store,Park,Dessert Shop,Japanese Restaurant,Coffee Shop,Restaurant
17,North York,2,Chinese Restaurant,Middle Eastern Restaurant,Caribbean Restaurant,Japanese Restaurant,Supermarket,Sushi Restaurant,Korean Restaurant,Bakery,Hotel,Dessert Shop
18,North York,2,Caribbean Restaurant,Bakery,Chinese Restaurant,Middle Eastern Restaurant,Supermarket,Sushi Restaurant,Grocery Store,Park,Korean Restaurant,Japanese Restaurant
19,North York,2,Coffee Shop,Middle Eastern Restaurant,Bakery,Japanese Restaurant,Chinese Restaurant,Bagel Shop,Park,Dessert Shop,Supermarket,Korean Restaurant
20,North York,2,Café,Park,Middle Eastern Restaurant,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bakery,Liquor Store,Korean Restaurant,Supermarket
21,North York,2,Coffee Shop,Bagel Shop,Japanese Restaurant,Middle Eastern Restaurant,Supermarket,Bakery,Park,Dessert Shop,Caribbean Restaurant,Liquor Store
22,North York,2,Supermarket,Bagel Shop,Coffee Shop,Middle Eastern Restaurant,Park,Café,Furniture / Home Store,Bakery,Korean Restaurant,BBQ Joint
23,North York,2,Café,Coffee Shop,Park,Italian Restaurant,Middle Eastern Restaurant,Liquor Store,Clothing Store,Restaurant,Korean Restaurant,Supermarket


#### Cluster 4

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,3,Park,Coffee Shop,Café,Grocery Store,Brewery,Beach,Liquor Store,Caribbean Restaurant,Bakery,Farmers Market
9,Scarborough,3,Park,Coffee Shop,Café,Beach,Brewery,Farmers Market,Grocery Store,Bakery,Pizza Place,Caribbean Restaurant
36,East York,3,Park,Coffee Shop,Café,Farmers Market,Beach,Brewery,Historic Site,BBQ Joint,Botanical Garden,Spa
37,East Toronto,3,Park,Coffee Shop,Café,Beach,Farmers Market,Brewery,Historic Site,Liquor Store,Bakery,Comic Shop
38,East York,3,Park,Café,Coffee Shop,Bakery,Ice Cream Shop,Italian Restaurant,Brewery,Gastropub,Diner,Sandwich Place
39,East York,3,Park,Coffee Shop,Café,Bakery,Italian Restaurant,Brewery,Historic Site,Farmers Market,Hotel,Gastropub
40,East York,3,Park,Coffee Shop,Farmers Market,Café,Historic Site,Brewery,Hotel,Beach,Gastropub,Ice Cream Shop
41,East Toronto,3,Park,Coffee Shop,Café,Hotel,Farmers Market,Historic Site,Restaurant,Brewery,Ice Cream Shop,Liquor Store
42,East Toronto,3,Coffee Shop,Park,Café,Farmers Market,Gastropub,Beach,Bakery,Brewery,Historic Site,BBQ Joint
43,East Toronto,3,Coffee Shop,Park,Café,Farmers Market,Brewery,Hotel,Gastropub,Historic Site,Beach,Japanese Restaurant


#### Cluster 5

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,North York,4,Coffee Shop,Café,Italian Restaurant,Park,Grocery Store,Liquor Store,Clothing Store,Steakhouse,Bakery,Restaurant
31,North York,4,Coffee Shop,Café,Italian Restaurant,Park,Grocery Store,Liquor Store,Clothing Store,Steakhouse,Bakery,Restaurant
32,North York,4,Coffee Shop,Café,Italian Restaurant,Park,Grocery Store,Liquor Store,Clothing Store,Steakhouse,Bakery,Restaurant
33,North York,4,Coffee Shop,Café,Italian Restaurant,Park,Grocery Store,Liquor Store,Clothing Store,Steakhouse,Bakery,Restaurant
86,Mississauga,4,Bakery,Indian Restaurant,Grocery Store,Park,Liquor Store,Burrito Place,Restaurant,Breakfast Spot,Sporting Goods Shop,Coffee Shop
89,Etobicoke,4,Coffee Shop,Park,Bakery,Burger Joint,Seafood Restaurant,Liquor Store,Grocery Store,Pizza Place,Sporting Goods Shop,Ice Cream Shop
94,Etobicoke,4,Park,Coffee Shop,Bakery,Café,Restaurant,Burger Joint,Grocery Store,Ice Cream Shop,Sandwich Place,Sporting Goods Shop
95,Etobicoke,4,Bakery,Burger Joint,Grocery Store,Seafood Restaurant,Coffee Shop,Liquor Store,Japanese Restaurant,Sporting Goods Shop,Restaurant,Eastern European Restaurant
96,North York,4,Italian Restaurant,Hotel,Liquor Store,Restaurant,Bakery,Steakhouse,Sporting Goods Shop,Sushi Restaurant,Supermarket,Asian Restaurant
97,North York,4,Hotel,Indian Restaurant,Chinese Restaurant,Italian Restaurant,Steakhouse,Restaurant,Grocery Store,Coffee Shop,Burger Joint,Ice Cream Shop


## On New York

The same method as above will be applied on New York City.  We have called here the neighborhood dataframe "neighborhoods". We will change this dataframe name to df_newyork with which we will extract the venues using Foursquare API.  Then we will segment and classify into simular clusters. 

In [113]:
# For clarity let rename the New York neighborhood geospatial dataframe as df_newyork 
df_newyork = neighborhoods

In [114]:
df_newyork.head(12)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


Find the name of the first neighborhood

In [115]:
neighborhood_name = df_newyork.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'Wakefield'.


Find geospatial coordinates of Wakefield

In [116]:
neighborhood_latitude = df_newyork.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_newyork.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


 Let's get the top 1000 venues that are in The Beaches within a radius of 10000 meters.

In [117]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

Extracting venues categories

In [118]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning Json dataframe

In [119]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875
1,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935
2,Anna Artuso's Pastry Shop,Bakery,40.902991,-73.865839
3,Rory Dolan's,Bar,40.904888,-73.868792
4,Fish & Ting,Caribbean Restaurant,40.885656,-73.829197
5,Avitable Bros. Deli,Deli / Bodega,40.902441,-73.863791
6,The Heritage,Pub,40.903309,-73.865991
7,Van Cortlandt Park - John Muir Nature Trail,Trail,40.899504,-73.873027
8,Gourmet Express,Café,40.890508,-73.817591
9,Foodtown,Grocery Store,40.912845,-73.835658


### Exploring New York Venues

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Looping the function on each venue to generate a new dataframe named "newyork_venues"

In [122]:
newyork_venues = getNearbyVenues(names=df_newyork['Neighborhood'],
                                   latitudes=df_newyork['Latitude'],
                                   longitudes=df_newyork['Longitude']
                                  )

newyork_venues.head(12)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
5,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
6,Wakefield,40.894705,-73.847201,Subway,40.890468,-73.849152,Sandwich Place
7,Wakefield,40.894705,-73.847201,Central Deli,40.896728,-73.844387,Deli / Bodega
8,Wakefield,40.894705,-73.847201,Louis Pizza,40.898399,-73.848810,Pizza Place
9,Wakefield,40.894705,-73.847201,Koss Quick Wash,40.891281,-73.849904,Laundromat


In [123]:
# Total Number of venues
newyork_venues.shape

(10049, 7)

Number of venues returned for each Neighborhood

In [124]:
newyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,29,29,29,29,29,29
Annadale,9,9,9,9,9,9
Arden Heights,4,4,4,4,4,4
Arlington,5,5,5,5,5,5
Arrochar,21,21,21,21,21,21
Arverne,19,19,19,19,19,19
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,20,20,20,20,20,20


Number of unique categories can be curated from all the returned venues

In [125]:
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 431 uniques categories.


431 uniques categories of venues were found for New York vs 233 in Toronto

### Analysis of similarity in each New York Neighborhood

In [126]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

,Zoo Exhibit,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other G

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [127]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.head(12)

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other G

Checking the 12 most common venues in each neighborhood

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nyneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
nyneighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    nyneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

nyneighborhoods_venues_sorted.head(12)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Chinese Restaurant,Department Store,Spa,Grocery Store,Fast Food Restaurant,Check Cashing Service,Bus Station
1,Annadale,Pizza Place,Dance Studio,Pharmacy,Liquor Store,Food,Train Station,Diner,Restaurant,Farmers Market,Financial or Legal Service
2,Arden Heights,Rental Car Location,Coffee Shop,Pizza Place,Pharmacy,Yoga Studio,Fish Market,Factory,Falafel Restaurant,Farm,Farmers Market
3,Arlington,Deli / Bodega,Coffee Shop,Bus Stop,American Restaurant,Intersection,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Outdoors & Recreation,Polish Restaurant,Mediterranean Restaurant,Bagel Shop,Taco Place,Liquor Store,Sandwich Place
5,Arverne,Surf Spot,Sandwich Place,Metro Station,Restaurant,Pizza Place,Playground,Coffee Shop,Donut Shop,Board Shop,Beach
6,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Seafood Restaurant,Bakery,Mediterranean Restaurant,Pizza Place,Indian Restaurant,Café
7,Astoria Heights,Burger Joint,Bowling Alley,Chinese Restaurant,Bakery,Gourmet Shop,Pizza Place,Hostel,Playground,Plaza,Deli / Bodega
8,Auburndale,Italian Restaurant,Korean Restaurant,Mattress Store,Fast Food Restaurant,Pet Store,Pharmacy,Miscellaneous Shop,Toy / Game Store,Mobile Phone Shop,Train
9,Bath Beach,Chinese Restaurant,Pizza Place,Italian Restaurant,Cantonese Restaurant,Pharmacy,Donut Shop,Bubble Tea Shop,Gas Station,Fast Food Restaurant,Coffee Shop


### Clustering venues in New York Neighborhoods

Running k-Means algorithm to group neighborhood into clusters. Let take k = 5

In [130]:
# set number of clusters = 5
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
New dataframe of top 12 venues for each neighborhood

In [131]:
# add clustering labels
nyneighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

newyork_merged = df_newyork

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(nyneighborhoods_venues_sorted.set_index('Neighborhood'), on ='Neighborhood')

newyork_merged.head(12) # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Deli / Bodega,Sandwich Place,Dessert Shop,Gas Station,Ice Cream Shop,Donut Shop,Laundromat,Pizza Place,Filipino Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,0,Bus Station,Bagel Shop,Accessories Store,Deli / Bodega,Fried Chicken Joint,Grocery Store,Discount Store,Restaurant,Pharmacy,Basketball Court
2,Bronx,Eastchester,40.887556,-73.827806,0,Diner,Deli / Bodega,Bus Stop,Caribbean Restaurant,Bus Station,Metro Station,Convenience Store,Chinese Restaurant,Seafood Restaurant,Juice Bar
3,Bronx,Fieldston,40.895437,-73.905643,0,River,Bus Station,Business Service,Plaza,Yoga Studio,Fishing Spot,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,0,Bus Station,Park,Gym,Bank,Food Truck,Locksmith,Plaza,Yoga Studio,Farm,Farmers Market
5,Bronx,Kingsbridge,40.881687,-73.902818,0,Pizza Place,Bar,Bakery,Latin American Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Shipping Store,Spanish Restaurant
6,Manhattan,Marble Hill,40.876551,-73.910660,0,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Department Store,Diner,Discount Store,Donut Shop,Pizza Place,Seafood Restaurant
7,Bronx,Woodlawn,40.898273,-73.867315,0,Deli / Bodega,Pub,Playground,Pizza Place,Train Station,Rental Car Location,Cosmetics Shop,Donut Shop,Pharmacy,Park
8,Bronx,Norwood,40.877224,-73.879391,0,Pizza Place,Bank,Park,Pharmacy,Burger Joint,Grocery Store,Coffee Shop,Bus Station,Fast Food Restaurant,Mobile Phone Shop
9,Bronx,Williamsbridge,40.881039,-73.857446,0,Bar,Nightclub,Soup Place,Caribbean Restaurant,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


#### Let's visualize the resulting clusters¶

In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        newyork_merged['Latitude'], 
        newyork_merged['Longitude'], 
        newyork_merged['Neighborhood'], 
        newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clustering of New York Neighborhood venues

#### Ctuster 1

In [133]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Deli / Bodega,Sandwich Place,Dessert Shop,Gas Station,Ice Cream Shop,Donut Shop,Laundromat,Pizza Place,Filipino Restaurant
1,Co-op City,Bus Station,Bagel Shop,Accessories Store,Deli / Bodega,Fried Chicken Joint,Grocery Store,Discount Store,Restaurant,Pharmacy,Basketball Court
2,Eastchester,Diner,Deli / Bodega,Bus Stop,Caribbean Restaurant,Bus Station,Metro Station,Convenience Store,Chinese Restaurant,Seafood Restaurant,Juice Bar
3,Fieldston,River,Bus Station,Business Service,Plaza,Yoga Studio,Fishing Spot,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Riverdale,Bus Station,Park,Gym,Bank,Food Truck,Locksmith,Plaza,Yoga Studio,Farm,Farmers Market
5,Kingsbridge,Pizza Place,Bar,Bakery,Latin American Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Shipping Store,Spanish Restaurant
6,Marble Hill,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Department Store,Diner,Discount Store,Donut Shop,Pizza Place,Seafood Restaurant
7,Woodlawn,Deli / Bodega,Pub,Playground,Pizza Place,Train Station,Rental Car Location,Cosmetics Shop,Donut Shop,Pharmacy,Park
8,Norwood,Pizza Place,Bank,Park,Pharmacy,Burger Joint,Grocery Store,Coffee Shop,Bus Station,Fast Food Restaurant,Mobile Phone Shop
9,Williamsbridge,Bar,Nightclub,Soup Place,Caribbean Restaurant,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


#### Ctuster 2

In [134]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Yoga Studio,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service
203,Todt Hill,Park,Yoga Studio,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service


#### Ctuster 3

In [135]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
179,Neponsit,Beach,Yoga Studio,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service


#### Ctuster 4

In [136]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
207,Port Ivory,Bar,Flea Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service


#### Ctuster 5

In [137]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
255,Emerson Hill,Construction & Landscaping,Yoga Studio,Flower Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service
303,Bayswater,Playground,Construction & Landscaping,Yoga Studio,Fishing Spot,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field



## Results and Discussion <a name="results"></a>

The natural diversity of cities is always trivial.  The analysis of category of venues has shown that New York neighborhoods contain at least 431 uniques categories, while Toronto neighborhoods have only 233 categories.  This was clearly expected due to the fact that New York has been characterized as the world's premier financial center (Business Insider, Inc. 2014).

k-Means clustering method applied, in the same conditions, to the venues explored from neighborhoods of New York and Toronto into 10 most common clusters has shown different level of convergence.  New York clustering conververged at a third iteration while Toronto iterations went over a fifth rounds.  It trended that New York has far more diverse groups of business that Toronto. 

The results could be interpreted that for a qualified job seekers New york could be a better option because of multiple opportunities that Toronto, assuming that all political and public administrative influences were neglected.  But for small and medium enterprises investors, the method shows that Toronto could be a better option, considering that competition level in Toronto should be less agressive than in New York City.


## Conclusion <a name="conclusion"></a>

The analysis conducted here was conclusive assuming that all other factors: immigration policies, political, public administration etc, have no influence. This general comparison of business between New York and Toronto indicated the point was not to look at a certain "size" or "graduation" bu to explore the preponderance of groups of activities on on either sides.


## References <a name="references"></a>

[1] Toronto---Wikipedia

[2] New York ---Wikipedia

[3] Foursquare API

[4] "Top 8 Cities by GDP: China vs. The U.S." Business Insider, Inc. July 31, 2011. Retrieved July 29, 2014.